In [7]:
import numpy as np
from scipy.sparse import spdiags
import matplotlib.pyplot as plt

m = 64 # N value in x and y directions
n = m * m # total size of matrix
L = 20
dx = dy = L / m

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones
e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector
for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]
e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]
# Place diagonal elements

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offset = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
matA = spdiags(diagonals, offset, n, n).toarray() / (dx**2)
matA[0,0] = 2 / (dx**2)

A = matA

e1 = np.ones((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.ones((n,1))  
e4 = np.ones((n, 1 ))  


diagonals_B = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_B = [-n+m, -m, m , n-m]  # Below and above neighbors (one step in y-direction)

matB = spdiags(diagonals_B, offsets_B, n, n).toarray() / ( 2 * dx )

B = matB

e1 = np.zeros((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.copy(e2)  
e4 = np.copy(e1)  
for index in range(64):
    e1[64 * index]= 1
    e2[64 * index + 63] = 0
    e3[64 * index] = 0 
    e4[64 * index + 63]= 1


diagonals_C = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_C = [-m+1, -1, 1 , m-1]  # Below and above neighbors (one step in y-direction)

matC = spdiags(diagonals_C, offsets_C, n, n).toarray() / ( 2 * dy)


C = matC

import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp

# Parameters
nu = 0.001
Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

# Create the spatial grid
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

# Initial vorticity (elliptical Gaussian)
w = np.exp( - X ** 2 - (1/20) * (Y**2))
w2 = w.reshape((N))

kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Poisson equation solver (in Fourier space)

#def pde(nx, ny, K, N, B, C, A, nu, w2):
def pde(t, w2, nx, ny, A, B, C,  K , nu):
    w = w2.reshape(nx , ny)
    wt = fft2(w)
    psit = - wt / K 
    psi = np.real(ifft2(psit)).reshape(nx * ny)
    psi_x = np.dot(B, psi)
    psi_y = np.dot(C, psi)
    wx = np.dot(B, w2)
    wy = np.dot(C, w2)
    rhs = nu * np.dot(A, w2) + (wx * psi_y) - (psi_x * wy)#.reshape(nx * ny)
    return rhs

# Initial conditions (flattened initial vorticity)

#wsol = solve_ivp(lambda t, w_flat: pde(t, w2, nx, ny, A, B, C, nu), [t[0], t[-1]], w2, t_eval=t, method='RK45')
wsol = solve_ivp(pde, [t[0], t[-1]], w2, t_eval=t , args = (nx, ny, A, B, C,  K , nu) , method = 'RK45')

A1 = wsol.y
A1

array([[2.50656748e-46, 3.59333831e-45, 1.89111814e-44, ...,
        1.90694953e-42, 4.81782237e-42, 1.14589809e-41],
       [1.17762859e-43, 6.53501838e-43, 2.61944442e-42, ...,
        1.58752937e-40, 3.55405439e-40, 7.44358759e-40],
       [4.55107657e-41, 1.93017295e-40, 6.43169265e-40, ...,
        2.48618419e-38, 5.06420518e-38, 9.67454248e-38],
       ...,
       [1.96785570e-38, 1.23565898e-37, 5.47361406e-37, ...,
        4.35329967e-35, 1.02107473e-34, 2.22183945e-34],
       [6.19028421e-41, 5.34413246e-40, 2.94014414e-39, ...,
        3.68054386e-37, 9.43670395e-37, 2.22914621e-36],
       [1.60178709e-43, 1.99725187e-42, 1.41305817e-41, ...,
        2.79956305e-39, 7.94959391e-39, 2.07092106e-38]])

In [5]:
# (b) 
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import bicgstab, gmres
from scipy.linalg import lu
from scipy.integrate import solve_ivp
import time

P, L, U = lu(A)
Pb = np.dot(P, b)
y = solve_triangular(L, Pb, lower=True)
x = solve_triangular(U, y)


NameError: name 'b' is not defined

In [10]:
# Part b : A \ b

import numpy as np
from scipy.sparse import spdiags
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import time 

m = 64 # N value in x and y directions
n = m * m # total size of matrix
L = 20
dx = dy = L / m

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones
e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector
for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one

# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]
e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]
# Place diagonal elements

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offset = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
matA = spdiags(diagonals, offset, n, n).toarray() / (dx**2)
matA[0,0] = 2 / (dx**2)

A = matA

e1 = np.ones((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.ones((n,1))  
e4 = np.ones((n, 1 ))  


diagonals_B = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_B = [-n+m, -m, m , n-m]  # Below and above neighbors (one step in y-direction)

matB = spdiags(diagonals_B, offsets_B, n, n).toarray() / ( 2 * dx )

B = matB

e1 = np.zeros((n, 1))  
e2 = np.ones((n, 1))  
e3 = np.copy(e2)  
e4 = np.copy(e1)  
for index in range(64):
    e1[64 * index]= 1
    e2[64 * index + 63] = 0
    e3[64 * index] = 0 
    e4[64 * index + 63]= 1


diagonals_C = [e1.flatten(), -e2.flatten(), e3.flatten(), -e4.flatten()]
offsets_C = [-m+1, -1, 1 , m-1]  # Below and above neighbors (one step in y-direction)

matC = spdiags(diagonals_C, offsets_C, n, n).toarray() / ( 2 * dy)


C = matC

nu = 0.001
Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

# Create the spatial grid
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

w = np.exp( - X ** 2 - (1/20) * (Y**2))
w2 = w.reshape((N))

def ab_rhs(t, w2, nx, ny, A, B, C,  nu):    
    #w2 = w.reshape(( nx, ny))
    psi = np.linalg.solve(A, w2)
    psi_x = np.dot(B, psi)
    psi_y = np.dot(C, psi)
    wx = np.dot(B, w2)
    wy = np.dot(C, w2)
    rhs = nu * np.dot(A, w.flatten()) - (wx * psi_y) + (psi_x * wy)
    return rhs
    
absol = solve_ivp(ab_rhs, [t[0], t[-1]], w2, t_eval=t , args = (nx, ny, A, B, C, nu) , method = 'RK45')

A2 = absol.y 

start_time = time.time() # Record the start time
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

A2

Elapsed time: 0.00 seconds


array([[ 2.50656748e-46,  1.46260636e-45,  2.67455597e-45, ...,
         7.75650821e-45,  1.01863698e-44,  5.29808991e-45],
       [ 1.17762859e-43, -2.17529770e-29,  1.79644906e-29, ...,
        -1.74645899e-19, -6.68217216e-19,  1.28619495e-18],
       [ 4.55107657e-41, -4.09230065e-29,  3.36874445e-29, ...,
        -3.16271586e-20, -2.82715063e-19,  1.18115669e-18],
       ...,
       [ 1.96785570e-38, -2.53146722e-29,  2.30465962e-29, ...,
        -2.81683828e-20, -7.38755489e-21, -3.14078467e-20],
       [ 6.19028421e-41,  8.78062311e-30, -4.21301499e-30, ...,
        -2.85507569e-20,  8.62770298e-20, -5.78808055e-19],
       [ 1.60178709e-43,  9.33161535e-30, -6.35111671e-30, ...,
         3.87116415e-20,  4.99321675e-19, -1.99440605e-18]])